In [1]:
from dataset import Dataset
import logging
import napari
import matplotlib.pyplot as plt
from segmentation import SegmentNuclei, SegmentCytoplasm

logging.basicConfig(level=logging.DEBUG)


d1 = Dataset(name="lumbar_1")
d1.load_nuclei()
d1.load_cytoplasm()
# d1.load_other_channel(channel="green")
d1.load_gene_expression()
df = d1.gene_expression

idx = list()
for i, gene in df.iterrows():
    if gene.x > 6000 and gene.x <= 8000:
        if gene.y > 5000 and gene.y <= 6000:
            idx.append(i)
df = df.iloc[idx]
df.x = df.x - 6000
df.y = df.y - 5000
d1.gene_expression = df

d1.images["Cytoplasm"] = d1.images["Cytoplasm"][5000:6000, 6000:8000]

d1.images["Nuclei"] = d1.images["Nuclei"][5000:6000, 6000:8000]

C:\Users\jetja\miniconda3\envs\scSpatial\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [2]:
seg = SegmentCytoplasm(size=120, flow_threshold=0.4, mask_threshold=0)
d1.run_segmentation(seg)

INFO:cellpose.core:>>>> using CPU
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 53.32 sec


In [3]:
seg.map_genes(d1)

In [4]:
d1.segmentation.gene_expression

value                                                      ...  \
gene      Adamts5 Aif1 Calb2 Cplx1 Cpne4 Crabp1 Csf1r Elavl4 Gad2 Gal  ...   
object_id                                                              ...   
0               8    1     2     3     0      0     4      1    0   0  ...   
1               0    0     0     0     1      0     1      0    0   0  ...   
3               0    0     0     0     0      0     0      1    0   0  ...   
4               0    0     0     0     1      0     0      0    0   0  ...   
5               0    0     0     0     0      0     0      0    0   0  ...   
6               0    0     0     0     0      0     0      0    0   0  ...   
7               0    1     0     0     0      0     0      0    0   0  ...   
8               0    0     0     0     0      0     0      0    0   0  ...   
12              0    1     0     1     0      0     0      0    1   0  ...   
13              0    1     0     0     0      0     0      0    0   0  ...   
14              0    0     0     0     1      0     0      0    0   0  ...   
16              1    1     0     0     0      0     0      0    0   0  ...   
17              0    0     0     0     1      0     0      0    0   0  ...   
19              0    0     0     0     0      0     0      0    1   0  ...   
20              0    0     0     0     0      0     0      1    0   0  ...   
23              3    1     0     0     0      0     0      0    0   0  ...   
25              0    1     0     0     0      0     1      1    0   0  ...   
30              0    1     0     0     0      0     0      0    0   0  ...   
31              0    0     0     0     0      0     0      0    0   0  ...   
32              0    0     0     0     0      0     0      0    0   0  ...   
33              0    0     0     0     0      0     0      0    0   0  ...   
34              0    0     0     2     0      1     0      0    0   0  ...   
35              0    0     0     0     0      0     1      0    1   0  ...   
36              0    0     0     0     0      0     0      0    0   0  ...   
37              0    1     0     0     0      0     0      0    0   0  ...   
38              0    0     0     0     0      0     0      1    0   0  ...   
42              1    0     0     0     0      0     0      0    0   0  ...   
46              0    0     0     0     1      0     0      0    0   1  ...   
48              0    0     0     0     0      0     0      0    0   0  ...   
49              0    0     0     0     0      1     0      0    0   0  ...   
50              0    0     0     0     0      0     0      0    0   0  ...   
51              0    0     1     1     0      1     1      1    0   0  ...   
56              0    0     0     0     0      0     0      0    1   0  ...   
59              0    0     0     1     0      0     0      0    0   0  ...   
60              0    0     0     0     0      0     0      0    0   0  ...   
62              0    0     0     0     1      0     0      0    0   0  ...   
63              0    0     0     0     0      0     0      0    0   0  ...   
65              0    0     0     0     1      0     0      0    0   0  ...   
68              0    0     0     0     0      0     0      0    0   0  ...   

                                                                    
gene      Npy2r Nts Pdyn Slc32a1 Slc6a5 Sorcs3 Spp1 Sstr2 Tac2 Trh  
object_id                                                           
0             0   0    0       0      0      4    1     0    0   3  
1             0   0    0       0      0      0    0     0    0   0  
3             0   0    0       0      0      0    0     0    0   0  
4             0   0    0       0      0      0    0     0    0   0  
5             0   0    0       0      0      0    1     0    0   0  
6             0   0    0       0      0      1    0     0    0   0  
7             0   0    0       0      0      0    0     0    0   0  
8             0   0    0       0      0      

In [61]:
import pandas as pd
gene_map = list()
for i, gene in d1.gene_expression.iterrows():
    object_id = d1.segmentation.objects[int(gene.PosY), int(gene.PosX)]
    gene_map.append((gene.Gene, object_id, 1))
df = pd.DataFrame(gene_map, columns=["gene", "object_id", "value"])
df.pivot_table(index="object_id", columns="gene", fill_value=0, aggfunc=sum)

value                                                      ...  \
gene      Adamts5 Aif1 Calb2 Cplx1 Cpne4 Crabp1 Csf1r Elavl4 Gad2 Gal  ...   
object_id                                                              ...   
0               8    1     2     3     0      0     4      1    0   0  ...   
1               0    0     0     0     1      0     1      0    0   0  ...   
3               0    0     0     0     0      0     0      1    0   0  ...   
4               0    0     0     0     1      0     0      0    0   0  ...   
5               0    0     0     0     0      0     0      0    0   0  ...   
6               0    0     0     0     0      0     0      0    0   0  ...   
7               0    1     0     0     0      0     0      0    0   0  ...   
8               0    0     0     0     0      0     0      0    0   0  ...   
12              0    1     0     1     0      0     0      0    1   0  ...   
13              0    1     0     0     0      0     0      0    0   0  ...   
14              0    0     0     0     1      0     0      0    0   0  ...   
16              1    1     0     0     0      0     0      0    0   0  ...   
17              0    0     0     0     1      0     0      0    0   0  ...   
19              0    0     0     0     0      0     0      0    1   0  ...   
20              0    0     0     0     0      0     0      1    0   0  ...   
23              3    1     0     0     0      0     0      0    0   0  ...   
25              0    1     0     0     0      0     1      1    0   0  ...   
30              0    1     0     0     0      0     0      0    0   0  ...   
31              0    0     0     0     0      0     0      0    0   0  ...   
32              0    0     0     0     0      0     0      0    0   0  ...   
33              0    0     0     0     0      0     0      0    0   0  ...   
34              0    0     0     2     0      1     0      0    0   0  ...   
35              0    0     0     0     0      0     1      0    1   0  ...   
36              0    0     0     0     0      0     0      0    0   0  ...   
37              0    1     0     0     0      0     0      0    0   0  ...   
38              0    0     0     0     0      0     0      1    0   0  ...   
42              1    0     0     0     0      0     0      0    0   0  ...   
46              0    0     0     0     1      0     0      0    0   1  ...   
48              0    0     0     0     0      0     0      0    0   0  ...   
49              0    0     0     0     0      1     0      0    0   0  ...   
50              0    0     0     0     0      0     0      0    0   0  ...   
51              0    0     1     1     0      1     1      1    0   0  ...   
56              0    0     0     0     0      0     0      0    1   0  ...   
59              0    0     0     1     0      0     0      0    0   0  ...   
60              0    0     0     0     0      0     0      0    0   0  ...   
62              0    0     0     0     1      0     0      0    0   0  ...   
63              0    0     0     0     0      0     0      0    0   0  ...   
65              0    0     0     0     1      0     0      0    0   0  ...   
68              0    0     0     0     0      0     0      0    0   0  ...   

                                                                    
gene      Npy2r Nts Pdyn Slc32a1 Slc6a5 Sorcs3 Spp1 Sstr2 Tac2 Trh  
object_id                                                           
0             0   0    0       0      0      4    1     0    0   3  
1             0   0    0       0      0      0    0     0    0   0  
3             0   0    0       0      0      0    0     0    0   0  
4             0   0    0       0      0      0    0     0    0   0  
5             0   0    0       0      0      0    1     0    0   0  
6             0   0    0       0      0      1    0     0    0   0  
7             0   0    0       0      0      0    0     0    0   0  
8             0   0    0       0      0      

In [ ]:
viewer = napari.Viewer(axis_labels=["Dorsoventral", "Mediolateral"])

viewer.add_image(d1.images["Nuclei"], name="Nuclei", colormap="yellow")
viewer.add_image(
    d1.images["Cytoplasm"], name="Cytoplasm", blending="additive", colormap="cyan"
)
viewer.add_points(data=list(zip(df.PosY, df.PosX)), properties=df, text="Gene")

viewer.add_labels(d1.segmentation.objects)